# Proyecto Final - Deployment

En el notebook presentado a continuación se presenta una simulación del deployment de los diferentes modelos creados en el Notebook principal denominado "Proyecto Final.ipynb"

## Carga de Paquetes

In [2]:
# Paquetes utilizados
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import collections
from pandas.api.types import is_integer_dtype, is_float_dtype
from sklearn import preprocessing
from sklearn import svm
from sklearn import tree

# Habilitar la compatibilidad con tensorflow v1 si se tienen tensorflow v2
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


## Feature Engineering

In [3]:
# Se guardan los datos del CSV en un dataframe de pandas
raw_dataset = pd.read_csv("data_titanic_proyecto.csv")

# ================================
# Separación de "Name" 
# ================================

# Se crea un "feature engineering" dataset
fe_dataset = raw_dataset.copy()

# Se separa la columna de "name" en "first_name" y "last_name"
# Se emplea la opción "expand" para crear dos columnas a partir de una
fe_dataset[["last_name", "first_name"]] = raw_dataset["Name"].str.split(", ", expand = True)

# Se extrae el honorífico del "first_name" utilizando el punto como un separador
fe_dataset[["honorific", "first_name"]] = fe_dataset["first_name"].str.split(".", expand = True)

# Elimina la columna de "name"
fe_dataset = fe_dataset.drop("Name", axis = 1)

# ================================
# Creación de "married"
# ================================

# Se obtiene el número de fila de todas las "mrs"
mrs = fe_dataset["honorific"] == "Mrs"

# Se extraen los nombres que contienen paréntesis
parenthesis = fe_dataset["first_name"].str.contains('(', regex = False)

# Este comando tiene dos partes
# Derecha: 
#   1. Se obtienen todos los "first_names" de aquellas "mrs" que contienen paréntesis. 
#   2. Se utiliza una expresión regular para separar el nombre de casada y soltera
#   3. La expresión genera 3 columnas nuevas "0" (copia de la siguiente), "maiden_name" y "single_name"
# Izquierda:
#   1. Se agregan las 3 nuevas columnas creadas por el regex al dataframe
#   2. Solo se agregan datos a las filas que son "mrs" con paréntesis en su nombre
fe_dataset.loc[mrs & parenthesis, ["temp", "married_name", "single_name"]] = fe_dataset.loc[mrs & parenthesis, "first_name"].str.extract('((?P<married_name>.+?) )?\((?P<single_name>.+)\)', expand = True)

# Se elimina la columna "temp" 
fe_dataset = fe_dataset.drop("temp", axis = 1)

# Se construye el nombre completo del esposo de cada Mrs
full_spouse_name = fe_dataset["married_name"].astype(str) + " " + fe_dataset["last_name"]

# Se eliminan las filas que contienen "nan"
# O mejor dicho, se mantienen solo las filas que no tienen "NaNs"
full_spouse_name = full_spouse_name[~full_spouse_name.str.contains("nan")]

# Se convierte el dataframe en una lista de strings
full_spouse_name = list(full_spouse_name)

# Se vuelven a construir los nombres completos 
full_name = fe_dataset["first_name"] + " " + fe_dataset["last_name"]

# Se elimina el texto adicional entre paréntesis de los nombres completos
full_name = full_name.str.replace(' \(.*\)', '', regex = True)

# Se elimina el texto adicional entre comillas
full_name = full_name.str.replace(' ".*"', '', regex = True)

# Se revisa si alguno de los nombres en "full_name" están en la
# lista de "full_spouse_name"
mr_married = full_name.isin(full_spouse_name)

# Se crea una nueva columna denominada "married"
# Inicialmente se asume que nadie está casado
fe_dataset["married"] = 0

# Se coloca como "casada" la persona que es una "mrs" (el honorífico indica esto)
# o que se comprobó que estaba casada con una "mrs"
fe_dataset.loc[mr_married | mrs, "married"] = 1

# Se eliminan las columnas sobrantes luego del proceso de determinar a los casados
# "first_name", "married_name" y "single_name".
fe_dataset = fe_dataset.drop(["first_name", "married_name", "single_name"], axis = 1)

# ================================
# Eliminación de columnas
# ================================

# Se elimina la columna de ID
fe_dataset = fe_dataset.drop("PassengerId", axis = 1)

# Se elimina la columna de Cabin
fe_dataset = fe_dataset.drop("Cabin", axis = 1)

# ================================
# Imputación de NaNs "Embarked"
# ================================

# Se obtiene la moda de "Embarked" y se extrae su valor 
# Luego se reemplazan todos los NaNs (isna()) por la moda anterior
fe_dataset.loc[fe_dataset["Embarked"].isna(), "Embarked"] = fe_dataset["Embarked"].mode().values[0]

# ================================
# Label encoding "class" y "survived"
# ================================

# Se crea el objeto para label encoding (LE) para cada columna
LE_class = preprocessing.LabelEncoder()
LE_survived = preprocessing.LabelEncoder()

# Se hace el label encoding para cada variable
fe_dataset["passenger_class_enc"] = LE_class.fit_transform(fe_dataset["passenger_class"])
fe_dataset["passenger_survived_enc"] = LE_survived.fit_transform(fe_dataset["passenger_survived"])

# ================================
# OHE "passenger_sex"
# ================================

# Se crea el objeto para one hot encoding (OHE) de cada columna
OHE_sex = preprocessing.OneHotEncoder()

# Antes de aplicar el encoding, la data debe ser redimensionada usando "array.reshape(-1,1)"
data_to_encode = fe_dataset["passenger_sex"].values.reshape(-1,1)

# Se codifica el array anterior y luego se convierte como tal a un array
# (ya que la salida se obtiene como una "sparse matrix")
data_matrix_enc = OHE_sex.fit_transform(data_to_encode).toarray()

# 1. Se convierte la matriz anterior en un dataframe con ints
# 2. Se agregan las columnas generadas al dataset
fe_dataset[["Female", "Male"]] = pd.DataFrame(data_matrix_enc, columns = ["Female", "Male"]).astype(int)

# Para análisis posteriores, se crea una columna con "one-hot-encoding k-1" para
# el sexo (básicamente lo mismo que la columna "Female").
fe_dataset["passenger_sex_enc"] = fe_dataset["passenger_sex"].astype('category')
fe_dataset["passenger_sex_enc"] = fe_dataset["passenger_sex_enc"].cat.codes

# =======================
# OHE "embarked"
# =======================

# Se crea el objeto para one hot encoding (OHE) de cada columna
OHE_embarked = preprocessing.OneHotEncoder()

# Antes de aplicar el encoding, la data debe ser redimensionada usando "array.reshape(-1,1)"
data_to_encode = fe_dataset["Embarked"].values.reshape(-1,1)

# Se codifica el array anterior y luego se convierte como tal a un array
# (ya que la salida se obtiene como una "sparse matrix")
data_matrix_enc = OHE_embarked.fit_transform(data_to_encode).toarray()

# 1. Se convierte la matriz anterior en un dataframe con ints
# 2. Se agregan las columnas generadas al dataset
fe_dataset[["C", "Q", "S"]] = pd.DataFrame(data_matrix_enc, columns = ["C", "Q", "S"]).astype(int)

# Para análisis posteriores, se crea una columna con label encoding para "Embarked"
fe_dataset["embarked_enc"] = fe_dataset["Embarked"].astype('category')
fe_dataset["embarked_enc"] = fe_dataset["embarked_enc"].cat.codes

# =======================
# Frequency Encoding "last_name" y "honorific"
# =======================

# Columnas a las que se le aplicará frequency encoding
freq_enc_columns = ["last_name", "honorific"]

# Se itera sobre cada columna para frequency encoding
for column in freq_enc_columns:

    # Función que obtiene el porcentaje de ocurrencia de cada categoría
    frequency_encode = fe_dataset.groupby(column).size() / len(fe_dataset)

    # Se asigna a cada categoría su porcentaje de ocurrencia
    fe_dataset[column + "_enc"] = fe_dataset[column].apply(lambda x: frequency_encode[x])

# =======================
# Fare_per_person y group_size
# =======================

# Se inicializan las nuevas columnas 
fe_dataset["fare_per_person"] = 0.0             # Se inicializa con floats para obtener decimales 
fe_dataset["group_size"] = 0                    # Se inicializa con ints para obtener enteros

# Se recorre cada fila en el dataset
for ind, row in raw_dataset.iterrows():
    
    # 1. Se extrae el número de ocurrencias para cada tipo de ticket
    # 2. Se extrae el tipo de ticket actual y se indexa el resultado anterior
    # 3. El resultado es el tamaño de grupo 
    group_size = raw_dataset["Ticket"].value_counts()[row["Ticket"]]

    # Se agrega el precio por persona y tamaño de grupo a cada columna
    fe_dataset.at[ind, "fare_per_person"] = row["Fare"] / group_size
    fe_dataset.at[ind, "group_size"] = group_size

# Se elimina la columna de fare previa
fe_dataset = fe_dataset.drop("Fare", axis = 1)

# =======================
# Ticket number y prefix
# =======================

# Se divide la variable de Ticket en un prefijo y un número
fe_dataset[["temp", "ticket_prefix", "ticket_number"]] = fe_dataset["Ticket"].str.extract("((?P<prefix>.*) )?(?P<number>\d*)", expand = True)

# La operación anterior genera una columna adicional que consiste 
# de una copia de "ticket_prefix". Se elimina esta columna
fe_dataset = fe_dataset.drop("temp", axis = 1)

# También se elimina la columna de "Ticket"
fe_dataset = fe_dataset.drop("Ticket", axis = 1)

# Se sustituye la categoría "nan" (sin prefijo) por "No Prefix"
fe_dataset.loc[fe_dataset["ticket_prefix"].isna(), "ticket_prefix"] = "No Prefix"

# Se convierte en un número la columna de "ticket_number"
fe_dataset["ticket_number_int"] = pd.to_numeric(fe_dataset["ticket_number"])

# Al convertir a número "ticket_number" habían algunas casillas sin número
# que se convirtieron en NaNs. Se igualan los NaNs a 0.
fe_dataset.loc[fe_dataset["ticket_number_int"].isna(), "ticket_number_int"] = 0

# Se hace label encoding del "ticket_prefix"
fe_dataset["ticket_prefix_enc"] = fe_dataset[column].astype('category')
fe_dataset["ticket_prefix_enc"] = fe_dataset["ticket_prefix_enc"].cat.codes

# =======================
# Imputación Age
# =======================

# 1. Se agrupan los datos según la clase 
# 2. Se rellenan los valores faltantes de todas las columnas con su media
# 3. Se guarda únicamente la columna "rellena" de edad
fe_dataset.loc[:, "Age"] = fe_dataset.groupby(["passenger_class"]).transform(lambda x: x.fillna(x.mean()))

# =======================
# Normalización
# =======================

# Columnas a normlizar
columns_to_normalize = ["SibSp", "Parch", "Age", "fare_per_person", "ticket_number_int"]

# Se itera sobre cada columna a normalizar
for column in columns_to_normalize:

    fe_dataset[column + "_norm"] = (fe_dataset[column] - fe_dataset[column].mean()) / fe_dataset[column].std()

## Selección de Variables

In [4]:
# Se seleccionan las columnas a utilizar
# Por el momento se excluye el ticket number porque cuenta con valores nulos
proc_dataset = fe_dataset[["Age", "Age_norm", "SibSp", "SibSp_norm", "Parch", "Parch_norm", "fare_per_person", "fare_per_person_norm", 
                           "C", "Q", "S", "embarked_enc", "passenger_class_enc", "passenger_sex_enc", "Male", "Female", "last_name_enc", 
                           "honorific_enc", "married", "ticket_prefix_enc", "ticket_number_int", "ticket_number_int_norm", "passenger_survived_enc"]]

# Se renombran las columnas
proc_dataset.columns = ["age", "age_norm", "sibsp", "sibsp_norm", "parch", "parch_norm", "fare_per_person", "fare_per_person_norm",
                        "C", "Q", "S", "embarked", "passenger_class", "passenger_sex", "male", "female", "last_name", "honorific", 
                        "married", "ticket_prefix", "ticket_number", "ticket_number_norm", "survived"]

## Declaración de Funciones

Función para predecir con Naive Bayes

In [5]:
def PredictNBC(data, model):

    # Se extraen los elementos de la tupla de "model"
    P_priori = model[0]
    dists = model[1]

    # Lista vacía de predicciones
    preds = []

    # Se inicializa la cuenta de predicciones correctas en 0
    correct = 0

    # Por cada fila en el dataframe "data"
    for index, row in data.iterrows():

        # Se inicializan las probabilidades por label
        P = dict()

        # Se extraen las features (todas las columnas menos la última) y el target
        x_pred = row.iloc[0:len(data.columns)-1]
        y_pred = row["survived"]

        # Por cada label en "survived"
        for label in P_priori.keys():

            # El valor inicial de la probabilidad de la label actual
            # es igual al valor de P(y) (probabilidad a priori)
            P[label] = P_priori[label]

            # Por cada columna en las features
            for col, val in x_pred.iteritems():

                # Si es una variable categórica
                if is_integer_dtype(data[col]):

                    # P(y|x1,x2,...,xn) = P(x1|y) * P(x2|y) * ...* P(xn|y) * P(y)
                    P[label] *= dists[label][col][val]

                # Si es una variable numérica
                elif is_float_dtype(data[col]):
                    
                    # P(y|x1,x2,...,xn) = P(x1|y) * P(x2|y) * ...* P(xn|y) * P(y)
                    P[label] *= dists[label][col](val)


        # Se realiza la predicción:
        # Se obtiene la label del valor con la mayor probabilidad
        pred = max(P, key = P.get)
        preds.append(pred) 

        # Se suma 1 a la cuenta de correctos si la predicción es igual al valor real
        if pred == y_pred:
            correct += 1

    # Se calcula el accuracy
    accuracy = correct / len(data)

    return preds, accuracy      

Función para predecir con Regresión Logística

In [6]:
def Predict(df, theta):

    from sklearn.preprocessing import PolynomialFeatures

    # =======================================
    # CREACIÓN DE GRAFO
    # =======================================

    # Dimensiones del modelo
    NumCols = theta.shape[0]
    NumCat = theta.shape[1]

    # Se extraen los datos "x" y "y" del dataframe
    x_test = df.iloc[:, 0:len(df.columns)-1]
    y_test = df["survived"]

    # Se convierten los dataframes en arrays de numpy
    x_test = x_test.to_numpy()
    y_test = y_test.to_numpy()

    # Se redimensiona el vector de labels
    y_test = np.reshape(y_test, (-1, 1))

    # Se le agrega una fila de unos a "train_x" convirtiendo los datos en un polinomio de grado 1
    poly = PolynomialFeatures(1)
    x_test = poly.fit_transform(x_test)

    # Se reinicia la creación del grafo creado
    tf.reset_default_graph()

    # Se crea un objeto de tipo grafo
    grafo = tf.Graph()

    # Se incluyen nodos dentro del grafo
    with grafo.as_default():

        # Input: Se definen los datos de entrenamiento
        # X_test: Tantas filas como imágenes (None para un tamaño variable). Tantas columnas como pixeles (+ 1 columna de unos)
        # Y_test: Tantas filas como imágenes (None para un tamaño variable). 1 columna
        X = tf.placeholder(tf.float32, [None, NumCols], "X")
        Y = tf.placeholder(tf.float32, [None, 1], "Y")

        # Input: Parámetros de regresión lineal
        # Tantas filas como columnas tiene X_test. Tantas columnas como categorías de regresión hay
        params = tf.placeholder(dtype="float", shape=[NumCols, NumCat], name="params")

        # Predicción de la salida dados los parámetros 
        # Dims Output: (NoMuestras, NoPixeles) x (NoPixeles, NoCategorias) = (NoMuestras, NoCategorias)
        with tf.name_scope("Predict"):
            
            # Se calculan los logits (m*X + B = Logits)
            logits = tf.matmul(X, params)

            # Se calcula manualmente el valor de la estimación usando sigmoid
            # (usado para el cálculo del accuracy)
            probs = tf.nn.sigmoid(logits)

            # Se convierten las probabilidades anteriores en una predicción
            Y_hat = tf.round(probs)

        # Cálculo del error por medio de entropía cruzada
        with tf.name_scope("Cross_Entropy"):
            
            # Cálculo de la entropía cruzada
            error = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y)

            # tf.nn.softmax_cross_entropy_with_logits retorna el "loss" de cada fila o muestra
            # Se promedian todas las filas para obtener el costo final
            error = tf.reduce_mean(error)

            # Incluir el error en la parte de "Scalars" de Tensorboard
            error_summary = tf.summary.scalar("Error", error)

        with tf.name_scope("Accuracy"):

            # Se chequea cuales predicciones son iguales a las labels reales (element-wise)
            correct_pred = tf.math.equal(Y_hat, Y)

            # Se suma el número de predicciones correctas y luego se divide dentro del número total de predicciones
            accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            
            # Se incluye el accuracy en la parte de "Scalars" de Tensorboard
            accuracy_summary = tf.summary.scalar("Accuracy", accuracy)

        # Inicializar variables globales
        init = tf.global_variables_initializer()

    # =======================================
    # EJECUCIÓN DE GRAFO
    # =======================================
    
    with tf.Session(graph = grafo) as sess:
        
        # Inicializa todas las variables de ser necesario
        tf.initialize_all_variables().run()

        # Inicializar el grafo
        sess.run(init)

        # Se definen los inputs del grafo
        inputs_grafo = {
            X : x_test,
            Y : y_test,
            params: Theta
        }

        # Se extraen los parámetros resultantes de la regresión y el error
        pred = Y_hat.eval(feed_dict=inputs_grafo)
        err = error.eval(feed_dict=inputs_grafo)
        acc = accuracy.eval(feed_dict=inputs_grafo)

    return pred, err, acc

## Carga de Modelos

In [8]:
import joblib
import dill as pickle

# Carga de los modelos de SVM y decision tree
DT_model = joblib.load('./Models/DT_randomstate=102,max_depth=1.pkl')
SVM_model = joblib.load('./Models/SVM_C=3,degree=2,gamma=0.1,coef0=0.pkl')

# Carga del modelo de naive bayes
with open('./Models/NB.pkl', 'rb') as f:
    NB_model = pickle.load(f)

# Carga del modelo de regresión logística
Theta = np.load("./Models/RL_lr=0.01,epochs=6000,batch_size=128.npy", allow_pickle = True)

## Ensemble Learning

In [10]:
from scipy import stats

# Se vuelve a shufflear el processed dataset
proc_dataset.sample(frac = 1).reset_index(drop = True)

# Se obtiene una muestra con el 60% de las filas
ensemble_dataset = proc_dataset.sample(frac=0.6)

# 1. Se obtiene la "ground truth" (columna de survived)
# 2. Se convierte en un array de numpy
# 3. Se le hace un reshape para convertirlo en un vector columna
ground_truth = ensemble_dataset["survived"].to_numpy()
ground_truth = np.reshape(ground_truth, (-1,1))

# ===============================
# Decision Tree
# ===============================

# Se extraen las variables de interés
DT_data = ensemble_dataset[["age", "sibsp", "parch", "fare_per_person", "C", "Q", "S", "passenger_class", "passenger_sex", "last_name", "honorific", "ticket_prefix",
                            "ticket_number", "married", "survived"]]

# Se separan las features y la variable objetivo
DT_x = DT_data.iloc[:, 0:len(DT_data.columns)-1]

# Se generan las predicciones con el modelo
# Tipo de output: Array (-1,)
DT_preds = DT_model.predict(DT_x)

# Reshape para convertirlo en vector columna
DT_preds = np.reshape(DT_preds, (-1,1))

# ===============================
# SVM
# ===============================

# Se extraen las variables de interés
SVM_data = ensemble_dataset[["age_norm", "sibsp_norm", "parch_norm", "fare_per_person_norm", "C", "Q", "S", "passenger_class", "passenger_sex", "last_name", "honorific", "ticket_prefix",
                              "ticket_number_norm", "married", "survived"]]

# Se separan las features y la variable objetivo
SVM_x = SVM_data.iloc[:, 0:len(SVM_data.columns)-1]

# Se generan las predicciones con el modelo
# Tipo de output: Array (-1,)
SVM_preds = SVM_model.predict(SVM_x)

# Reshape para convertirlo en vector columna
SVM_preds = np.reshape(SVM_preds, (-1,1))

# ===============================
# Naive Bayes
# ===============================

# Se extraen las variables de interés (mismas variables que para el SVM)
NB_data = SVM_data

# Se hace la predicción y se evalúa la precisión
# Tipo de output: Lista
NB_preds, NB_accuracy = PredictNBC(NB_data, NB_model)

# Se convierte en array y luego se le hace reshape
# para convertirlo en vector columna
NB_preds = np.reshape(np.array(NB_preds), (-1,1))

# ===============================
# Regresión Logística
# ===============================

# Se extraen las variables de interés (mismas variables que para el SVM)
RL_data = SVM_data

# Se hace la predicción y se evalúa la precisión
# Tipo de output: Array (-1, 1)
RL_preds, RL_error, RL_accuracy_test = Predict(RL_data, Theta)

# ===============================
# Votación Mayoritaria
# ===============================

# Se concatenan horizontalmente los resultados
Results = np.hstack((DT_preds, SVM_preds, NB_preds, RL_preds))

# Se obtiene la moda de cada fila (voto mayoritario)
Vote = stats.mode(Results, axis = 1)[0]

# Se evalúa la precisión
# 1. Se obtiene el número de valores iguales entre la predicción y la salida real
# 2. Se divide este número dentro del número de muestras del dataset de validación
E_accuracy = list(Vote == ground_truth).count(True) / len(ground_truth)

# Se imprime la precisión de ensemble
print("Ensemble Accuracy Score:", "%.4f" % round(E_accuracy, 4))

Ensemble Accuracy Score: 0.8206
